Imports

In [46]:
import numpy as np
import tensorflow as tf
import os
import requests
import random
import logging
from music21 import converter, midi

Data loading


In [47]:
dataset_url = 'https://abc.sourceforge.net/NMD/nmd/reelsr-t.txt'  
local_file_path = 'data/tunes.txt'

os.makedirs(os.path.dirname(local_file_path), exist_ok=True)

response = requests.get(dataset_url, allow_redirects=True)
if response.status_code == 200:
    with open(local_file_path, 'wb') as file:
        file.write(response.content)

if os.path.exists(local_file_path) and os.path.getsize(local_file_path) > 0:
    print("Success")

Success


Data preprocessing

In [63]:
with open(local_file_path, 'r', encoding='utf-8') as file:
    tunes = file.read()

moods = ["happy", "sad", "calm", "energetic"]

def preprocess_tunes(abc):
    abc = abc.strip().split("\n")
    abc = [line for line in abc if not (line.startswith("%") or line.startswith("X:") or line.startswith("T:") or line.startswith("S:"))]
    abc.append(f"Mood: {random.choice(moods)}") 
    return "\n".join(abc)

tune_list = tunes.split("\n\n\n") 
tune_list = [preprocess_tunes(tune) for tune in tune_list]  

In [66]:
mood_labels = []
clean_tunes = []

for tune in tune_list:
    parts = tune.split("Mood:", 1)
    if len(parts) == 2:
        clean_tunes.append(parts[0].strip())  
        mood_labels.append(parts[1].strip())  

joined_tunes = "\n\n".join(clean_tunes)
vocab = sorted(set(joined_tunes))
vocab_len = len(vocab)

word_to_index = {char: i for i, char in enumerate(vocab)}
index_to_word = {i: char for char, i in word_to_index.items()}
mood_to_index = {mood: i for i, mood in enumerate(set(mood_labels))}
index_to_mood = {i: mood for mood, i in mood_to_index.items()}



Train model

In [67]:
sequence_length = 25 
x_train, y_train, mood_train = [], [], []

tunes_input = np.array([word_to_index[char] for char in joined_tunes], dtype=np.int32).reshape(-1, 1)

for i in range(sequence_length, len(tunes_input) - 1):
    x_train.append(tunes_input[i-sequence_length:i, 0])
    y_train.append(tunes_input[i, 0])
    mood_train.append(mood_to_index[mood_labels[i % len(mood_labels)]])

x_train = np.array(x_train)
y_train = np.array(y_train)
mood_train = np.array(mood_train)

mood_embedding_size = 10

mood_input = tf.keras.layers.Input(shape=(1,))
mood_embedding = tf.keras.layers.Embedding(len(mood_to_index), mood_embedding_size)(mood_input)
mood_flatten = tf.keras.layers.Flatten()(mood_embedding)

note_input = tf.keras.layers.Input(shape=(sequence_length, 1))
lstm_layer = tf.keras.layers.LSTM(512, return_sequences=True)(note_input)
lstm_layer = tf.keras.layers.LSTM(512)(lstm_layer)

merged = tf.keras.layers.concatenate([lstm_layer, mood_flatten])
dense_layer = tf.keras.layers.Dense(256, activation='relu')(merged)
output_layer = tf.keras.layers.Dense(vocab_len, activation='softmax')(dense_layer)

model = tf.keras.Model(inputs=[note_input, mood_input], outputs=output_layer)
model.compile(loss='sparse_categorical_crossentropy', optimizer='adam')

train_model = True

if train_model:
    model.fit([x_train, mood_train], y_train, epochs=1, batch_size=32)
    model.save('music_generator.h5')


1257/1257 ━━━━━━━━━━━━━━━━━━━━ 135s 105ms/step - loss: 2.0294


2025-03-10 20:19:29,377 - WARNING - You are saving your model as an HDF5 file via `model.save()` or `keras.saving.save_model(model)`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')` or `keras.saving.save_model(model, 'my_model.keras')`. 


Make predicitons amd generate

In [ ]:
import random

def generate_music(mood="happy", length=200):
    mood_index = np.array([[mood_to_index.get(mood, 0)]])
    seed_text = random.choice(clean_tunes)
    if len(seed_text) < sequence_length:
        seed_text = seed_text.ljust(sequence_length)
    else:
        seed_text = seed_text[-sequence_length:]

    seed_input = [word_to_index.get(char, 0) for char in seed_text]
    seed_input = np.array(seed_input).reshape(1, sequence_length, 1)

    generated_tune = seed_text

    for _ in range(length):
        prediction = model.predict([seed_input, mood_index])
        next_index = np.argmax(prediction)
        next_char = index_to_word[next_index]
        generated_tune += next_char
        seed_input = np.append(seed_input[:, 1:, :], [[[next_index]]], axis=1)

    mood_settings = {
        "happy": {"tempo": 100, "note_length": "1/8", "time_sig": "6/8"},
        "energetic": {"tempo": 120, "note_length": "1/8", "time_sig": "4/4"},
        "calm": {"tempo": 70, "note_length": "1/4", "time_sig": "3/4"},
        "sad": {"tempo": 70, "note_length": "1/2", "time_sig": "3/4"},
    }

    settings = mood_settings.get(mood, mood_settings["happy"])

    formatted_abc = f"""
X:1
T:Generated Tune ({mood.capitalize()})
M:{settings["time_sig"]}
L:{settings["note_length"]}
Q:{settings["tempo"]}
K:C
{generated_tune}
"""

    return formatted_abc.strip()


mood_input = "happy"
generated_abc = generate_music(mood_input)


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 198ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 176ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 48ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 36ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step
1/1 ━━━━━━

Output result

In [ ]:
from music21 import converter, midi
import pygame

def save_generated_music(abc_notation, filename="generated_tune.mid"):
    abc_stream = converter.parse(abc_notation, format='abc')
    midi_fp = filename
    mf = midi.translate.music21ObjectToMidiFile(abc_stream)
    mf.open(midi_fp, 'wb')        
    mf.write()
    mf.close()
        
def play_midi(midi_file):
    mf = midi.MidiFile()
    mf.open(midi_file)
    mf.read()
    mf.close()
    music_stream = midi.translate.midiFileToStream(mf)
    music_stream.show('midi')

save_generated_music(generated_abc)
play_midi("generated_tune.mid")

